In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from pathlib import Path

def stitch_two_images(img1, img2):
    
    # Grayscale
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    
    # Detecting SIFT features
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(gray1, None)
    kp2, des2 = sift.detectAndCompute(gray2, None)
    
    
    # Matching features
    matcher = cv2.BFMatcher()
    raw_matches = matcher.knnMatch(des1, des2, k=2)
   
    
    # Filtering matches using Lowe's ratio test
    good_matches = []
    for m, n in raw_matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)
    
    print(f"Found {len(good_matches)} good matches")
    
    if len(good_matches) < 4:
        raise Exception("Not enough good matches for homography")
    
    # Extracting location of good matches
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    
    # Finding homography matrix
    H, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    
    if H is None:
        raise Exception("Could not find homography")
    
    H = H.astype(np.float32)
    
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    
    
    # Image2
    corners2 = np.array([
        [0, 0],
        [0, h2-1],
        [w2-1, h2-1],
        [w2-1, 0]
    ], dtype=np.float32).reshape(-1, 1, 2)
    
    corners2_transformed = cv2.perspectiveTransform(corners2, H)
        
    # Image1
    corners1 = np.array([
        [0, 0],
        [0, h1-1],
        [w1-1, h1-1],
        [w1-1, 0]
    ], dtype=np.float32).reshape(-1, 1, 2)
    
    
    all_corners = np.concatenate((corners1, corners2_transformed), axis=0)
    x_min, y_min = np.int32(all_corners.min(axis=0).ravel())
    x_max, y_max = np.int32(all_corners.max(axis=0).ravel())
    
    
    translation = np.array([
        [1, 0, -x_min],
        [0, 1, -y_min],
        [0, 0, 1]
    ], dtype=np.float32)
    
    # Combined transformation
    H_with_translation = translation @ H
    
    output_width = x_max - x_min + 1
    output_height = y_max - y_min + 1
    output_shape = (output_width, output_height)
    
    warped_img2 = cv2.warpPerspective(img2, H_with_translation, output_shape)
    
    warped_img1 = cv2.warpPerspective(img1, translation, output_shape)
    
    
    # Creating masks for blending
    mask1 = (warped_img1 > 0).astype(np.uint8)
    mask2 = (warped_img2 > 0).astype(np.uint8)
    
    
    # Creating a weight map for blending
    overlap = mask1 * mask2
    weight1 = np.zeros_like(warped_img1, dtype=np.float32)
    weight2 = np.zeros_like(warped_img2, dtype=np.float32)
    
    for y in range(output_height):
        for x in range(output_width):
            if overlap[y, x].any():
                weight1[y, x] = 1 - (x / output_width)
                weight2[y, x] = x / output_width
            else:
                if mask1[y, x].any():
                    weight1[y, x] = 1
                if mask2[y, x].any():
                    weight2[y, x] = 1
    
    # Normalizing weights
    weight_sum = weight1 + weight2
    weight_sum[weight_sum == 0] = 1  
    weight1 /= weight_sum
    weight2 /= weight_sum
    
    
    # Blending the images
    result = np.zeros_like(warped_img1, dtype=np.float32)
    for c in range(3):  # Blending each channel separately
        result[:, :, c] = (warped_img1[:, :, c] * weight1[:, :, c] +
                           warped_img2[:, :, c] * weight2[:, :, c])
    
    result = np.clip(result, 0, 255).astype(np.uint8)
    
    
    # Creating visualizations
    output_dir = Path("panorama_output")
    output_dir.mkdir(exist_ok=True)
    
    # Saving visualization of the keypoint matches
    match_img = cv2.drawMatches(img1, kp1, img2, kp2, good_matches[:50], None, 
                              flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
    cv2.imwrite(str(output_dir / "matches_detail.jpg"), match_img)
    
    # Saving individual keypoint visualizations
    kp_img1 = cv2.drawKeypoints(img1, kp1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    kp_img2 = cv2.drawKeypoints(img2, kp2, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    cv2.imwrite(str(output_dir / "keypoints_img1.jpg"), kp_img1)
    cv2.imwrite(str(output_dir / "keypoints_img2.jpg"), kp_img2)
    
    # Saving intermediate warped images
    cv2.imwrite(str(output_dir / "warped_img1.jpg"), warped_img1)
    cv2.imwrite(str(output_dir / "warped_img2.jpg"), warped_img2)
    
    return result



In [3]:
def main():
    output_dir = Path("final_output")
    output_dir.mkdir(exist_ok=True)
    
    
    image_paths = [
        "images2/panorama12.jpg",
        "images2/panorama34.jpg",
    ]
    
    valid_paths = []
    for path in image_paths:
        if os.path.exists(path):
            valid_paths.append(path)
        else:
            print(f"Warning: Image file {path} not found")
    
    if len(valid_paths) < 2:
        print("Need at least 2 valid images for stitching")
        return
    
    
    images = []
    for path in valid_paths:
        img = cv2.imread(path)
        if img is None:
            print(f"Error loading image: {path}")
            continue
        images.append(img)
    
    if len(images) < 2:
        print("Need at least 2 valid images for stitching")
        return
    
    try:
        print("Stitching first two images...")
        panorama = stitch_two_images(images[0], images[1])
        cv2.imwrite(str(output_dir / "panorama.jpg"), panorama)
        print(f"Saved panorama to {output_dir / 'panorama.jpg'}")
        
        result = panorama
        for i in range(2, len(images)):
            print(f"Stitching image {i+1} to the panorama...")
            result = stitch_two_images(result, images[i])
            cv2.imwrite(str(output_dir / f"panorama_with_{i+1}_images.jpg"), result)
            print(f"Saved updated panorama to {output_dir / f'panorama_with_{i+1}_images.jpg'}")
        
        fig, axes = plt.subplots(1, len(images), figsize=(15, 5))
        for i, img in enumerate(images):
            if len(images) > 1:
                ax = axes[i]
            else:
                ax = axes
            ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            ax.axis('off')
            ax.set_title(f"Input {i+1}")
        
        plt.tight_layout()
        plt.savefig(output_dir / "input_images.png")
        plt.close()
        
        plt.figure(figsize=(15, 10))
        plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title("Final Panorama")
        plt.savefig(output_dir / "final_panorama_visualization.png")
        plt.close()
        
    except Exception as e:
        print(f"Error in panorama stitching: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Stitching first two images...
Found 278 good matches
Saved panorama to final_output\panorama.jpg
